In [1]:
!source activate twitterCOVID

In [8]:
from tweepy import OAuthHandler, AppAuthHandler
from tweepy import API, TweepError
from secrets import *
from textblob import TextBlob
import os
import json
import dataset
from datafreeze import freeze
import pandas as pd

# Consumer key authentication
auth = AppAuthHandler(consumer_key, consumer_secret) #using appauthhandler to retreive at a faster rate

# Access key authentication
# auth.set_access_token(access_token, access_token_secret)

# Set up the API with the authentication handler
api = API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify = True)

## Problem: SearchAPI is limited in many ways (max 100 tweets per query is just one of them)

In [9]:
res = api.search(q = "covid", until = "2020-04-02")

In [10]:
jsonres = [resi._json for resi in res]

In [11]:
results = pd.DataFrame(jsonres)

In [12]:
results

created_at                   id               id_str  \
0   Wed Apr 01 23:59:59 +0000 2020  1245501195554369536  1245501195554369536   
1   Wed Apr 01 23:59:59 +0000 2020  1245501195373948929  1245501195373948929   
2   Wed Apr 01 23:59:59 +0000 2020  1245501195327848449  1245501195327848449   
3   Wed Apr 01 23:59:59 +0000 2020  1245501195315220480  1245501195315220480   
4   Wed Apr 01 23:59:59 +0000 2020  1245501195273285632  1245501195273285632   
5   Wed Apr 01 23:59:59 +0000 2020  1245501195248156673  1245501195248156673   
6   Wed Apr 01 23:59:59 +0000 2020  1245501195210260480  1245501195210260480   
7   Wed Apr 01 23:59:59 +0000 2020  1245501195004907521  1245501195004907521   
8   Wed Apr 01 23:59:59 +0000 2020  1245501194900045824  1245501194900045824   
9   Wed Apr 01 23:59:59 +0000 2020  1245501194782429185  1245501194782429185   
10  Wed Apr 01 23:59:59 +0000 2020  1245501194736480256  1245501194736480256   
11  Wed Apr 01 23:59:59 +0000 2020  1245501194564444173  1245501194564444173   
12  Wed Apr 01 23:59:59 +0000 2020  1245501194488832001  1245501194488832001   
13  Wed Apr 01 23:59:59 +0000 2020  1245501194388176896  1245501194388176896   
14  Wed Apr 01 23:59:59 +0000 2020  1245501194317041672  1245501194317041672   

                                                 text  truncated  \
0   RT @DonaldJTrumpJr: The same media wants to st...      False   
1   RT @Econ_Marshall: Whatever happened to the Se...      False   
2   If I had a dollar..." COVID edition:  If I had...       True   
3   @look_now Trotzdem können sie die Krankheit ve...       True   
4   RT @RexChapman: Sandra the orangutang started ...      False   
5   Se preguntarán ¿De dónde es este pasillo de ho...       True   
6   RT @changsikkie_: Hiii!!!! Not COVID related, ...      False   
7   On 3/26, @emergesmarter hosted their 1st live ...       True   
8   RT @KavehAkbar: I have family in Iran who’ve d...      False   
9   RT @annecurtissmith: People are getting sick, ...      False   
10  Forced to move your courses online? Here are s...      False   
11  RT @HaddadDebochado: 🚨URGENTE: Covid-19 é um p...      False   
12  RT @adriana_donati: Shameful to hide this from...      False   
13  RT @moui: กษัตริย์มาเลเซีย และ ราชินีมาเลเซีย ...      False   
14  In recent years, laboratories on the continent...       True   

                                             entities  \
0   {'hashtags': [], 'symbols': [], 'user_mentions...   
1   {'hashtags': [], 'symbols': [], 'user_mentions...   
2   {'hashtags': [], 'symbols': [], 'user_mentions...   
3   {'hashtags': [], 'symbols': [], 'user_mentions...   
4   {'hashtags': [], 'symbols': [], 'user_mentions...   
5   {'hashtags': [], 'symbols': [], 'user_mentions...   
6   {'hashtags': [], 'symbols': [], 'user_mentions...   
7   {'hashtags': [], 'symbols': [], 'user_mentions...   
8   {'hashtags': [], 'symbols': [], 'user_mentions...   
9   {'hashtags': [], 'symbols': [], 'user_mentions...   
10  {'hashtags': [{'text': 'OnlineTeaching', 'indi...   
11  {'hashtags': [], 'symbols': [], 'user_mentions...   
12  {'hashtags': [{'text': 'WuhanCoronaVirus', 'in...   
13  {'hashtags': [], 'symbols': [], 'user_mentions...   
14  {'hashtags': [], 'symbols': [], 'user_mentions...   

                                             metadata  \
0   {'iso_language_code': 'en', 'result_type': 're...   
1   {'iso_language_code': 'en', 'result_type': 're...   
2   {'iso_language_code': 'en', 'result_type': 're...   
3   {'iso_language_code': 'de', 'result_type': 're...   
4   {'iso_language_code': 'en', 'result_type': 're...   
5   {'iso_language_code': 'es', 'result_type': 're...   
6   {'iso_language_code': 'en', 'result_type': 're...   
7   {'iso_language_code': 'en', 'result_type': 're...   
8   {'iso_language_code': 'en', 'result_type': 're...   
9   {'iso_language_code': 'en', 'result_type': 're...   
10  {'iso_language_code': 'en', 'result_type': 're...   
11  {'iso_language_code': 'pt', 'result_type': 're...   


## Cracking Twitter APIs

From [link](https://bhaskarvk.github.io/2015/01/how-to-use-twitters-search-rest-api-most-effectively./). The seacrh method has some `max_id` and `since_id` arguments we can exploit to our advantage. These are like upper and lower boundaries for our tweets, and given tweets come sorted by post time, we can retrieve many many tweets going back up to 1 week.

In [28]:
import sys
import jsonpickle
import os

searchQuery = "covid"
maxTweets = 1000
tweetsPerQry = 100 # max set by twitter api
fName = "LastWeek.csv"

# the api search output SearchObject is an object that can be turned into a list of json str with the _json method
# the search object itself can be indexed like a list
sinceID = None # no lower limit (as far as necessary)
maxID = -1 #exploit negative indexing, last tweet
tweetCount = 0
# the ID is just an ordered number, If I tweet now and my tweet ID is 1 and
# your tweet comes next your tweet id will be 2


print("Downloading max {0} tweets".format(maxTweets))

with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            if (maxID<=0):
                # if your maxID is negative (1st iteration) do this
                if (not sinceID):
                    new_tweets = api.search(q=searchQuery, count = tweetsPerQry)
                else: # if sinceID exists set sinceID as that minimum
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since_id=sinceID)
            else: # if maxID is positive ? 
                if (not sinceID): # if sinceID not defined don't set low boundary
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(maxID - 1))
                else: # if sinceID exist set low and up boundary
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(maxID - 1),
                                            since_id=sinceID)
            if not new_tweets:
                print("No more tweets found")
                break
            for tweet in new_tweets:
                f.write(jsonpickle.encode(tweet._json))
            tweetCount += len(new_tweets)
            print("Downloaded {0} tweets".format(tweetCount))
            maxID = new_tweets[-1].id # set maxID to the latest ID in the previous search
        except TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break
print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

1
Downloaded 100 tweets
1246251926246711296
3
Downloaded 200 tweets
1246251915131805696
3
Downloaded 300 tweets
1246251904738504705
3
Downloaded 400 tweets
1246251895078993920
3
Downloaded 500 tweets
1246251883859251200
3
Downloaded 600 tweets
1246251872773603333
3
Downloaded 700 tweets
1246251863592316928
3
Downloaded 800 tweets
1246251854268416003
3
Downloaded 900 tweets
1246251842281058311
3
Downloaded 1000 tweets
1246251831988236289
Downloaded 1000 tweets, Saved to LastWeek.csv
